In [1]:
import pandas as pd
import numpy as np

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

from matplotlib import pyplot as plt
import seaborn as sns

import graphviz
import pydotplus
import io

from scipy import misc

from sklearn.metrics import accuracy_score

import spotipy
import spotipy.util as util
sp = spotipy.Spotify() 
from spotipy.oauth2 import SpotifyClientCredentials 

%matplotlib inline
%matplotlib notebook

## Establishing a connection with Spotify API

I'm using a python wrapper around the Spotify API called spotipy. By signing in to the Spotify API and creating a new app, Spotify triggers a client ID and a client secret. We use these two keys to establish a connection. We then describe the scope of our access, that is the nature of our access.

In [2]:
cid ="09c567afd5784427b568ce3994c57bbc" 
secret = "626ee57c77b540a882e77568b96cf647"
username = ""
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
scope = 'user-library-read playlist-read-private'
token = util.prompt_for_user_token(username, scope='user-library-read playlist-read-private', client_id=cid,\
                                   client_secret=secret, \
                                   redirect_uri='https://localhost:8888/callback/')
if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)


In [3]:
# my personal spotify user_id and playlist_id for two playlists

abd_id = '22zwjgjvsa7o3q6himwitxuoy'
playlist_id_desi = '4bk3qrpg7jLfY9luOzqAky'
playlist_id_abd = '7shUpGkYn0lOchTXXzDk4E'
playlist_dislike = "0VfCY7JcZtG0P66SEHnMhr"

In [4]:
# fetching the songs from my first playlist

sourcePlaylist = sp.user_playlist(abd_id, playlist_id_abd)
tracks = sourcePlaylist["tracks"]
songs = tracks["items"] 
while tracks['next']:
    tracks = sp.next(tracks)
    for item in tracks["items"]:
        songs.append(item)

In [5]:
songs[0]['track']

{'album': {'album_type': 'single',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2oSONSC9zQ4UonDKnLqksx'},
    'href': 'https://api.spotify.com/v1/artists/2oSONSC9zQ4UonDKnLqksx',
    'id': '2oSONSC9zQ4UonDKnLqksx',
    'name': 'Atif Aslam',
    'type': 'artist',
    'uri': 'spotify:artist:2oSONSC9zQ4UonDKnLqksx'}],
  'available_markets': ['AD',
   'AE',
   'AR',
   'AT',
   'AU',
   'BE',
   'BG',
   'BH',
   'BO',
   'BR',
   'CA',
   'CH',
   'CL',
   'CO',
   'CR',
   'CY',
   'CZ',
   'DE',
   'DK',
   'DO',
   'DZ',
   'EC',
   'EE',
   'EG',
   'ES',
   'FI',
   'FR',
   'GB',
   'GR',
   'GT',
   'HK',
   'HN',
   'HU',
   'ID',
   'IE',
   'IL',
   'IN',
   'IS',
   'IT',
   'JO',
   'JP',
   'KW',
   'LB',
   'LI',
   'LT',
   'LU',
   'LV',
   'MA',
   'MC',
   'MT',
   'MX',
   'MY',
   'NI',
   'NL',
   'NO',
   'NZ',
   'OM',
   'PA',
   'PE',
   'PH',
   'PL',
   'PT',
   'PY',
   'QA',
   'RO',
   'SA',
   'SE',
   'SG',
   'SK',
   'SV',


In [27]:
#fetching the songs from my second playlist

sourcePlaylist_desi = sp.user_playlist(abd_id, playlist_id_desi)
tracks_desi = sourcePlaylist_desi["tracks"]
songs_desi = tracks_desi["items"] 
while tracks_desi['next']:
    tracks_desi = sp.next(tracks_desi)
    for item in tracks_desi["items"]:
        songs_desi.append(item)

In [12]:
songs_desi[0]

{'added_at': '2018-03-24T18:19:53Z',
 'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/226p6bwbtmsimqipqol55zgei'},
  'href': 'https://api.spotify.com/v1/users/226p6bwbtmsimqipqol55zgei',
  'id': '226p6bwbtmsimqipqol55zgei',
  'type': 'user',
  'uri': 'spotify:user:226p6bwbtmsimqipqol55zgei'},
 'is_local': False,
 'primary_color': None,
 'track': {'album': {'album_type': 'single',
   'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0LyfQWJT6nXafLPZqxe9Of'},
     'href': 'https://api.spotify.com/v1/artists/0LyfQWJT6nXafLPZqxe9Of',
     'id': '0LyfQWJT6nXafLPZqxe9Of',
     'name': 'Various Artists',
     'type': 'artist',
     'uri': 'spotify:artist:0LyfQWJT6nXafLPZqxe9Of'}],
   'available_markets': ['AD',
    'AE',
    'AR',
    'AT',
    'AU',
    'BE',
    'BG',
    'BH',
    'BO',
    'BR',
    'CA',
    'CH',
    'CL',
    'CO',
    'CR',
    'CY',
    'CZ',
    'DE',
    'DK',
    'DO',
    'DZ',
    'EC',
    'EE',
    'EG',
    'E

In [26]:
sourcePlaylist_dislike = sp.user_playlist(abd_id, playlist_dislike)
tracks_dislike = sourcePlaylist_dislike["tracks"]
songs_dislike = tracks_dislike["items"] 
while tracks_dislike['next']:
    tracks_dislike = sp.next(tracks_dislike)
    for item in tracks_dislike["items"]:
        songs_dislike.append(item_dislike)

In [38]:
#storing the name and id of the unique songs in seperate lists because it makes it easier
#for me to fetch them when I use SQL queries

good_ids = []
good_names = []

bad_names = []
bad_ids = []

'''for i in range(len(songs_desi)):
    if (songs_desi[i]['track']['id'] not in good_ids) and (songs_desi[i]['track']['id'] not in songs_ids):
        good_ids.append(songs_desi[i]['track']['id'])
        good_names.append(songs_desi[i]['track']['name'])'''
for i in range(len(songs)):
    if (songs[i]['track']['id'] not in good_ids) and (songs[i]['track']['id'] not in songs_ids):
        good_ids.append(songs[i]['track']['id'])
        good_names.append(songs[i]['track']['name'])
for i in range(len(songs_dislike)):
    if (songs_dislike[i]['track']['id'] not in bad_ids) and (songs_dislike[i]['track']['id'] not in songs_ids):
        bad_ids.append(songs_dislike[i]['track']['id'])
        bad_names.append(songs_dislike[i]['track']['name'])

In [39]:
len(good_ids)

2

In [48]:
good_names[0]

'Dil Beparvah - The Dewarists, Season 5'

In [40]:
#for each song, I'm fetching its audio features as measured by the Spotify API
#and appending them in the features list. Each element is a dictionary with the audio attributes

features = []
for i in range(0,len(good_ids),50):
    audio_features = sp.audio_features(good_ids[i:i+50])
    for track in audio_features:
        features.append(track)
        features[-1]['label'] = 1
        
for i in range(0,len(bad_ids),50):
    audio_features = sp.audio_features(bad_ids[i:i+50])
    for track in audio_features:
        features.append(track)
        features[-1]['label'] = 0

In [41]:
audio_analysis_1 = sp.audio_analysis(good_ids[0])

In [42]:
audio_analysis_1.keys()

dict_keys(['meta', 'track', 'bars', 'beats', 'tatums', 'sections', 'segments'])

In [43]:
audio_analysis_1['sections']

[{'start': 0.0,
  'duration': 6.96101,
  'confidence': 1.0,
  'loudness': -23.453,
  'tempo': 137.478,
  'tempo_confidence': 0.781,
  'key': 1,
  'key_confidence': 0.0,
  'mode': 1,
  'mode_confidence': 0.0,
  'time_signature': 3,
  'time_signature_confidence': 1.0},
 {'start': 6.96101,
  'duration': 13.47067,
  'confidence': 0.024,
  'loudness': -19.04,
  'tempo': 137.75,
  'tempo_confidence': 0.775,
  'key': 1,
  'key_confidence': 0.0,
  'mode': 1,
  'mode_confidence': 0.0,
  'time_signature': 3,
  'time_signature_confidence': 1.0},
 {'start': 20.43168,
  'duration': 32.61528,
  'confidence': 0.753,
  'loudness': -15.631,
  'tempo': 138.133,
  'tempo_confidence': 0.655,
  'key': 6,
  'key_confidence': 0.012,
  'mode': 1,
  'mode_confidence': 0.274,
  'time_signature': 4,
  'time_signature_confidence': 0.214},
 {'start': 53.04696,
  'duration': 16.07682,
  'confidence': 0.284,
  'loudness': -12.864,
  'tempo': 138.178,
  'tempo_confidence': 0.625,
  'key': 1,
  'key_confidence': 0.044

In [10]:
features[-6]

{'danceability': 0.289,
 'energy': 0.983,
 'key': 2,
 'loudness': -4.589,
 'mode': 1,
 'speechiness': 0.112,
 'acousticness': 6.98e-06,
 'instrumentalness': 0.000445,
 'liveness': 0.278,
 'valence': 0.132,
 'tempo': 93.993,
 'type': 'audio_features',
 'id': '7KKlC664xSI3m453ZFuvLQ',
 'uri': 'spotify:track:7KKlC664xSI3m453ZFuvLQ',
 'track_href': 'https://api.spotify.com/v1/tracks/7KKlC664xSI3m453ZFuvLQ',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/7KKlC664xSI3m453ZFuvLQ',
 'duration_ms': 209827,
 'time_signature': 3,
 'label': 0}

## Connecting to MySQL

In [5]:
import pymysql as mdb

con = mdb.connect(host = 'localhost', 
                  user = 'root', 
                  passwd = 'abdullah1439', 
                  charset='utf8', use_unicode=True);

In [6]:
# Query to create a database
db_name = 'spotify_songs'
create_db_query = "CREATE DATABASE IF NOT EXISTS {db} DEFAULT CHARACTER SET 'utf8'".format(db=db_name)

# Create a database
cursor = con.cursor()
cursor.execute(create_db_query)
cursor.close()

/Users/macbook/Documents/anaconda3/lib/python3.7/site-packages/pymysql/cursors.py:170: Warning: (1007, "Can't create database 'spotify_songs'; database exists")
  result = self._query(query)


In [9]:


cursor = con.cursor()
table_name = 'all_songs'
create_table_query = '''CREATE TABLE IF NOT EXISTS {db}.{table} 
                                (ind int,
                                song_name varchar(250), 
                                song_id varchar(250), 
                                danceability float,
                                energy float,
                                `key` int,
                                loudness float,
                                mode int,
                                speechiness float,
                                acousticness float,
                                instrumentalness float,
                                liveness float,
                                valence float,
                                tempo float,
                                label int,
                                PRIMARY KEY(song_id)
                                )'''.format(db=db_name, table=table_name)
cursor.execute(create_table_query)
cursor.close()

In [ ]:
query_template = '''INSERT IGNORE INTO {db}.{table}(ind,
                                            song_name, 
                                            song_id, 
                                            danceability, 
                                            energy, 
                                            `key`,
                                            loudness,
                                            mode,
                                            speechiness,
                                            acousticness,
                                            instrumentalness,
                                            liveness,
                                            valence,
                                            tempo) 
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'''.format(db=db_name, table=table_name)
cursor = con.cursor()

for i in range(len(features)):
    ind = i
    song_name = good_names[i]
    song_id = good_ids[i]
    danceability = features[i]['danceability']
    energy = features[i]['energy']
    s_key = features[i]['key']
    loudness = features[i]['loudness']
    mode = features[i]['mode']
    speechiness = features[i]['speechiness']
    acousticness = features[i]['acousticness']
    instrumentalness = features[i]['instrumentalness']
    liveness = features[i]['liveness']
    valence = features[i]['valence']
    tempo = features[i]['tempo']
    
    print("Inserting station", song_name, "with song id: ", song_id)
    query_parameters = (ind, song_name, song_id, danceability, energy, s_key, loudness, mode, speechiness, acousticness, \
                       instrumentalness, liveness, valence, tempo)
    cursor.execute(query_template, query_parameters)

con.commit()
cursor.close()

In [7]:

cur = con.cursor(mdb.cursors.DictCursor)
cur.execute("SELECT * FROM {db}.{table}".format(db='spotify_songs', table='all_songs'))
rows = cur.fetchall()
cur.close()

songs_df = pd.DataFrame(list(rows))

In [8]:
songs_ids = list(songs_df["song_id"])

In [10]:
songs_names = list(songs_df["song_name"])

['Hum Hai Piya Ji Ke Patar Tiriywa',
 'Gham Lagata Ae Raja',
 'Kharihani Me',
 'Sorry',
 'Ae Dil Hai Mushkil Title Track',
 'Champions League Theme',
 'Bhar Jata Dhodi',
 'Only Human',
 'Odhni Ke Rang Piyar',
 'Hello',
 'Old Ways',
 'Shape of You',
 'LA CANCIÓN',
 'On The Road (feat. Meek Mill & Lil Baby)',
 'Laga Ke Fair Lovely',
 'Chokra Jawaan',
 'Raate Balamua Dehle Gari Ho, Tarkari Ke',
 'Umran Langiyaan',
 'Choliya Saket Hota Raja Ji',
 'Pyar Wali Baat Hokheda',
 'Let Me Love You',
 'Un Poco Loco',
 'Use This Gospel',
 'Talk',
 'Sapne',
 'Chor Bazari (From "Love Aaj Kal ")',
 'Señorita',
 'E Darling Kahiya Hoyi Milan',
 'Playing Games',
 'Kismat Se Nikal Ke Chal Jaibu',
 'Cazador',
 'Beneath Us - Original Mix',
 'Trigger',
 'Sau Aasmaan',
 'Tu Hi Re',
 'Khaabon Ke Parinday',
 'Bhatar Aiehe Holi Ke Baad',
 'Pagli Deewani',
 'Piyawa Se Pahile 2',
 'Aik Alif',
 'Tune Kaha',
 'Lose You To Love Me',
 "Don't",
 'Stay With Me',
 'A Whole New World (End Title) - From "Aladdin"',
 'Tu Bin

In [13]:
import lyricsgenius

In [14]:
genius = lyricsgenius.Genius("r3EhHdzkkCeFpV7MrR_cKvMuoIrkHPn9F0MBTcP6-_mBPPccca6jJrSeZKC_S0lt")

In [18]:
songs_names[-50:-20]

['How Do You Sleep?',
 'gone girl',
 'Aaja',
 "Maybe It's Time",
 'Wow.',
 'This Is What It Feels Like',
 'The Skylab Plan',
 'Achha Sila Diya Tune Mere Pyar Ka',
 'Dancing With A Stranger (with Normani)',
 'The Burden',
 'Bade Bhag Se Jodidar Jhalkauwa Milal Ba',
 'Stricken',
 'Despacito - Remix',
 'Chunariya Mein Daag Lag Gail',
 'Khalbali',
 'Wohi Khuda Hai',
 'Kab Aaoge Pardeshi Piya',
 'Meherbani',
 'Tempestades',
 'Gadbad Ghotala',
 'Fave Thing',
 'Maar Ke Aail Biya Ittar',
 "Don't Start Now",
 'Mountains',
 'Udd Gaye - Bacardi House Party Sessions',
 'Time',
 'Somebody',
 'Palani Me Jawani Rowata',
 'Yunhi Rastay Mai - From "7 Din Mohabbat In"',
 'Har Ek Baat Pe Kehte Ho Tum']

In [19]:
songs_lyrics = []
for _ in songs_names[-50:-20]:
    lyr = genius.search_song(_)
    if lyr:
        songs_lyrics.append(lyr.lyrics)
    else:
        songs_lyrics.append("N/A")

Searching for "How Do You Sleep?"...
Done.
Searching for "gone girl"...
Done.
Searching for "Aaja"...
Done.
Searching for "Maybe It's Time"...
Done.
Searching for "Wow."...
Done.
Searching for "This Is What It Feels Like"...
Done.
Searching for "The Skylab Plan"...
Done.
Searching for "Achha Sila Diya Tune Mere Pyar Ka"...
No results found for: 'Achha Sila Diya Tune Mere Pyar Ka'
Searching for "Dancing With A Stranger (with Normani)"...
Done.
Searching for "The Burden"...
Done.
Searching for "Bade Bhag Se Jodidar Jhalkauwa Milal Ba"...
No results found for: 'Bade Bhag Se Jodidar Jhalkauwa Milal Ba'
Searching for "Stricken"...
Done.
Searching for "Despacito - Remix"...
Done.
Searching for "Chunariya Mein Daag Lag Gail"...
No results found for: 'Chunariya Mein Daag Lag Gail'
Searching for "Khalbali"...
Done.
Searching for "Wohi Khuda Hai"...
Done.
Searching for "Kab Aaoge Pardeshi Piya"...
No results found for: 'Kab Aaoge Pardeshi Piya'
Searching for "Meherbani"...
Done.
Searching for "T

In [24]:
songs_lyrics[-4]

"[Intro]\nCome on\nNatalie La Rose\n\n[Chorus: Jeremih]\nI wanna rock with somebody\nI wanna take shots with somebody\nI wanna leave with somebody\nAnd we ain't gonna tell nobody, we ain't gonna tell nobody\n\n[Verse 1: Natalie La Rose]\nWalk in, everybody knows what's good\nBut some gonna hate, what's new?\nWe just do our thing, no time to waste\nAlways play it cool\n\n[Pre-Chorus: Natalie La Rose (Jeremih)]\nAnd at the end of the night when the lights go out\nWill we turn down? Oh no we won't\n(We ain't never turnin' down, we ain't never turnin' down)\nAnd when they try to make us leave\nWe turn and say we never going home\n(And you know just what I wanna do)\n\n[Chorus: Jeremih]\nI wanna rock with somebody\nI wanna take shots with somebody\nI wanna leave with somebody\nAnd we ain't gotta tell nobody, we ain't gotta tell nobody\n\n[Verse 2: Natalie La Rose]\nWalk in, everybody know what's good\nWe're our only plug\nBring us everything we think we need\nTonight is goin' up\nTonight is

In [15]:
NAMES = list(songs_df["song_name"])
songs_df[songs_df['label'] == 0].count()

acousticness        16
danceability        16
energy              16
ind                 16
instrumentalness    16
key                 16
label               16
liveness            16
loudness            16
mode                16
song_id             16
song_name           16
speechiness         16
tempo               16
valence             16
dtype: int64

In [ ]:
from vis import *

## Clustering

In [ ]:
from sklearn.preprocessing import StandardScaler

cluster_features = ['acousticness', 'danceability', 'instrumentalness', 'energy', 'speechiness']
df_cluster = songs_df[cluster_features]
X = np.array(df_cluster)
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

In [ ]:
X.shape

In [ ]:
from sklearn.cluster import KMeans

#Elbow method to determine k for k-means clustering

ss_dist = []
K = range(1, 11)
for k in K:
    km = KMeans(n_clusters=k, init='k-means++', random_state=123)
    km = km.fit(X)
    ss_dist.append(km.inertia_)
plt.figure()
plt.plot(K, ss_dist, 'bx-')
plt.xlabel('k')
plt.ylabel('Sum of squared distances')
plt.title('Elbow Method For Optimal k')
plt.show()

In [ ]:
df_cluster.hist(figsize=(10,5))
plt.show()

In [ ]:
num_clusters = 5
kmeanModel = KMeans(n_clusters=num_clusters, max_iter=10000, init='k-means++', random_state=123).fit(X)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=3, random_state=123)
pca_results = pca.fit_transform(X)

In [ ]:
print(pca.explained_variance_ratio_.sum())
pca.explained_variance_ratio_.cumsum()

In [ ]:
df_scree = pd.DataFrame({'Component': ['1','2','3'],'Indiv':pca.explained_variance_ratio_})
df_scree['cum_sum'] = df_scree['Indiv'].cumsum()
df_scree

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))
plt.bar(range(len(pca.explained_variance_ratio_)), pca.explained_variance_ratio_, 
        label='Individual', axes=ax, alpha=0.4)
plt.plot(range(len(pca.explained_variance_ratio_)), pca.explained_variance_ratio_.cumsum(), 
         label='Cumulative', color='tomato', axes=ax, marker='o')

ax.set_xticks(range(0,3))
ax.set_xticklabels(range(1,4), fontsize=12)
ax.set_yticklabels(range(0,90,10), fontsize=12)

plt.title('Scree Plot of PCA', fontsize=12)
plt.ylabel('Explained variance (%)', fontsize=12)
plt.xlabel('Principal components', fontsize=12)
plt.legend()
plt.show()

In [ ]:
df_pca = pd.DataFrame(pca_results)
df_pca.columns = ['PC1', 'PC2','PC3']
df_pca['label'] = kmeanModel.labels_
df_pca.head()

In [ ]:
plt.figure()
sns.set_style('white')
sns.scatterplot(data=df_pca, x='PC1', y='PC2', hue='label', style='label', palette='Set1')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title('Visualisation of Songs using PCA')
plt.show()

In [ ]:
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = Axes3D(fig)

ax.scatter(df_pca['PC1'], df_pca['PC2'], df_pca['PC3'],
           c=df_pca['label'], cmap='Set1')

ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')
plt.show()

In [ ]:
from sklearn.manifold import TSNE

# TSNE

tsne = TSNE(n_components=2, perplexity=50, n_iter=5000, random_state=123, learning_rate=190)
tsne_results = tsne.fit_transform(X)

In [ ]:
df_tsne = pd.DataFrame(tsne_results)
df_tsne.columns = ['D1', 'D2']
df_tsne['label'] = kmeanModel.labels_
df_tsne.head()

In [ ]:
NAMES = list(songs_df["song_name"])

In [ ]:
%matplotlib notebook

c = np.random.randint(1,5,size=len(NAMES))

norm = plt.Normalize(1,5)
cmap = plt.cm.RdYlGn

fig,ax = plt.subplots()
sc = plt.scatter(df_tsne['D1'], df_tsne['D2'], c=df_tsne['label'], s=70, cmap=cmap, norm=norm)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2)
plt.title('Visualisation of Songs using TSNE')
annot = ax.annotate("", xy=(0,0), xytext=(20,20),textcoords="offset points",
                    bbox=dict(boxstyle="round", fc="w"),
                    arrowprops=dict(arrowstyle="->"))
annot.set_visible(False)


def hover(event):
    def update_annot(ind):

        pos = sc.get_offsets()[ind["ind"][0]]
        annot.xy = pos
        text = "{}".format(" ".join([NAMES[n] for n in ind["ind"]]))
        annot.set_text(text)
        annot.get_bbox_patch().set_facecolor(cmap(norm(c[ind["ind"][0]])))
        annot.get_bbox_patch().set_alpha(0.4)


    vis = annot.get_visible()
    if event.inaxes == ax:
        cont, ind = sc.contains(event)
        if cont:
            update_annot(ind)
            annot.set_visible(True)
            fig.canvas.draw_idle()
        else:
            if vis:
                annot.set_visible(False)
                fig.canvas.draw_idle()
        

fig.canvas.mpl_connect("motion_notify_event", hover)
#plt.gcf().set_size_inches((12, 8)) 
plt.show();

In [ ]:
songs_df.loc[:,'cluster'] = kmeanModel.labels_
df_cluster_size = songs_df.groupby('cluster').size().reset_index(name='Num of songs')
plt.figure()
sns.barplot(data=df_cluster_size, x='cluster', y='Num of songs', color='seagreen')
plt.title('Distribution of clusters')
plt.show();

In [ ]:
# set binning intervals of 0.1
bins = np.linspace(0,1,10)

# create subplots
num_features = len(cluster_features)
f, axes = plt.subplots(num_clusters, num_features, 
                       figsize=(10, 5), sharex='col')

# initialise 1st row. Representing cluster
row = 0
for cluster in np.sort(songs_df['cluster'].unique()):
#     filter cluster dataframe and reset column to 0 for 1st plot for the cluster
    df_cluster = songs_df[songs_df['cluster'] == cluster]
    col = 0
    
    for feature in cluster_features:
#         create binned count for all recent df and clustered df
        rec_grp = songs_df.groupby(pd.cut(songs_df[feature], bins)).size().reset_index(name='count')
        cluster_grp = df_cluster.groupby(pd.cut(df_cluster[feature], bins)).size().reset_index(name='count')
        
#         plot overall distribution and cluster distribution on the ax
        sns.barplot(data=rec_grp, x=feature, y='count', 
                    color='grey', ax=axes[row, col])
        sns.barplot(data=cluster_grp, x=feature, y='count', 
                    color='red', ax=axes[row, col])
        
#         configure ax
        axes[row, col].set_xlabel('')
        axes[row, col].set_xticklabels(range(1,10), fontsize=12)
        if col > 0:
            axes[row, col].set_ylabel('')
        else:
            axes[row, col].set_ylabel('count', fontsize=12)
        if row == 0:
            axes[row, col].set_title(feature, fontsize=14)        
        col += 1
    row += 1
    
f.suptitle('Profile for each clusters')
    
plt.show()